# Main notebook

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Env Variables

In [3]:
DEFAULT_PLAYER_NB = 2
GRID_SIDE_SIZE = 25
MAX_POSITION = 624 # 25*25-1

## Environment Creation

### Tile

In [4]:
class Tile:
    """
    Description:
        Representation of a game tile. A tile is described by
        its color and its symbol. When a tile is not placed yet,
        it has a default state on the grid: "Blank" color and 
        "Empty" symbol.
        
    Parameters:
        color: (String) Color of the tile. Must be in the valid set of colors.
        
    """
    
    DEFAULT_COLOR = "Blank"
    DEFAULT_SYMBOL = "Empty"
    SET_OF_COLORS = {"Red","Blue","Green","Black","White"}
    SET_OF_SYMBOLS = {"Bird","Dog","Scarab","Scrib","Storage","Desert","Bonus"}
    SYMBOL_NUMBER_DICT = {"Bird":2,"Dog":2,"Scarab":2,"Scrib":1,"Storage":1,"Desert":1,"Bonus":2}
    
    
    def __init__(self,color=DEFAULT_COLOR,symbol=DEFAULT_SYMBOL):
        if color not in Tile.SET_OF_COLORS and color != Tile.DEFAULT_COLOR:
            raise ValueError
        if symbol not in Tile.SET_OF_SYMBOLS and symbol != Tile.DEFAULT_SYMBOL:
            raise ValueError
        self.color = color
        self.symbol = symbol
        
    @staticmethod
    def create_tiles_values_dict():
        """ Create the mapping between integers on the grid and the tiles. """
        tiles_values = {0:Tile(Tile.DEFAULT_COLOR,Tile.DEFAULT_SYMBOL)}
        colors = sorted(Tile.SET_OF_COLORS)
        symbols = sorted(Tile.SET_OF_SYMBOLS)
        i = 1
        for c in colors:
            for s in symbols:
                tiles_values[i] = Tile(c,s)
                i += 1
        return tiles_values

### Deck

In [5]:
class Deck:
    """
    Description:
        Representation of the two stack of Tiles from which the
        players can draw. The stacks are randomly shuffled according
        to the given seed.
        
    Parameters:
        seed: (int) Seed for initialization of the random number generator.
    
    """
    def __init__(self,seed=0):
        random.seed(seed)
        np.random.seed(seed)
        init_deck = []
        for c in Tile.SET_OF_COLORS:
            for s, n in Tile.SYMBOL_NUMBER_DICT.items():
                for i in range(n):
                    init_deck.append(Tile(c,s))
        random.shuffle(init_deck)
        self.stack_0 = init_deck[:len(init_deck)//2]
        self.stack_1 = init_deck[len(init_deck)//2:]
        
    def draw(self,stack):
        """ Draw a tile from the given stack. """
        if stack == 0:
            return self.stack_0.pop(0)
        else:
            return self.stack_1.pop(1)
        
    def tile_available(self):
        """ Return a tuple of the two available tiles'color. """
        Tile_0 = None
        Tile_1 = None
        if len(self.stack_0) != 0:
            Tile_0 = self.stack_0[0].color
        if len(self.stack_1) != 0:
            Tile_1 = self.stack_1[0].color
        return Tile_0, Tile_1

### Token

In [6]:
class Token():
    """
    Description:
        Representation of the token needed to buy
        tiles.
    
    Parameters:
        color: (String) Color of the token. Must be
                        included in "Red","Green","Blue",
                        "Black","White" or "Ankh".
    
    """
    
    REGULAR_COLORS = {"Red","Green","Blue","Black","White"}
    SPECIAL_COLOR = "Ankh"
    
    def __init__(self,color):
        if color not in Token.REGULAR_COLORS and color != Token.SPECIAL_COLOR:
            raise ValueError
        self.color = color

### Ressource Pool

In [24]:
class Ressource_Pool():
    """
    Description:
        Representation of the pool of tokens needed to buy
        tiles. Each stack of token is an integer.
        
    Parameters:
        players_number: (int) Number of player in the game.
    
    """
    
    def __init__(self,init_token_nb,max_regular_nb,max_special_nb,max_tot_reg_nb):
        self.MAX_REGULAR_NB = max_regular_nb
        self.MAX_SPECIAL_NB = max_special_nb
        self.MAX_REG_POOL_SIZE = max_tot_reg_nb
        self.red_tokens = [Token("Red") for i in range(init_token_nb)]
        self.green_tokens = [Token("Green") for i in range(init_token_nb)]
        self.blue_tokens = [Token("Blue") for i in range(init_token_nb)]
        self.black_tokens = [Token("Black") for i in range(init_token_nb)]
        self.white_tokens = [Token("White") for i in range(init_token_nb)]
        self.ankh_tokens = [Token("Amkh") for i in range(init_token_nb)]
        self.pool_dict = {"Red":self.red_tokens,"Green":self.green_tokens,
                           "Blue":self.blue_tokens,"Black":self.black_tokens,
                           "White":self.white_tokens,"Ankh":self.ankh_tokens}
        
    
    def fill(self, color):
        """ Add a token of the given color to the ressource pool. """
        color_pool = self.pool_dict[color]
        max_color_pool_size = (self.MAX_SPECIAL_NB if color == Token.SPECIAL_COLOR
                                    else self.MAX_REGULAR_NB)
        total_regular_pool_size = sum([len(self.pool_dict[c]) for c in Token.REGULAR_COLORS])
        if len(color_pool) >= max_color_pool_size:
            raise Exception(f"Maximum number of token reached for the color {color}")
        elif color != Token.SPECIAL_COLOR and total_regular_pool_size >= self.MAX_REG_POOL_SIZE:
            raise Exception(f"Maximum number of total token reached: {total_regular_pool_size}")
        else:
            color_pool.append(Token(color))
        
    def draw(self, color):
        """ Draw a token of the given color to the ressource pool. """
        if len(self.pool_dict[color]) <= 0:
            raise Exception(f"No token left for the color {color}")
        else:
            self.pool_dict[color].pop(0)

In [25]:
r = Ressource_Pool(0,5,2,5)

In [32]:
r.fill("Ankh")

In [34]:
r.ankh_tokens[1].color

'Ankh'

### Shop

In [ ]:
class Shop:
    
    def __init__(self,seed=0):
        self.deck = Deck(seed)
        self

### Global Environment

In [ ]:
class Ankhor_Env:
    def __init__(self,player_nb=DEFAULT_PLAYER_NB):
        self.player_nb = player_nb
        self.grid = np.zeros((GRID_SIDE_SIZE,GRID_SIDE_SIZE,player_nb))
        self.end = False
        self.curr_player = 0
        self.tiles_values = TILES_VALUES_DICT
        self.winner = None
        self.tile_stack = None
        
    def position_coordinates(position):
        """ Retrieve the coordinate on the grid given the integer position. """
        return (position//GRID_SIDE_SIZE,position%GRID_SIDE_SIZE)
    
    def check_basis(self,x,y,player,tile):
        """ Check if the tile can be placed on the relative basis. """
        neighbours = []
        for i in [-1,1]:
            for j in [-1,1]:
                neighbour_tile = self.grid[x+i,y+j,player]
                if neighbour_tile == 0: # Miss a tile in the basis
                    return False
                neighbours.append(neighbour_tile)
        basis_colors = [t.color for t in neighbours]
        return tile.color in basis_colors
                
        
    def check_valid(self,position,player,tile):
        """ Check if the move is valid. """
        if position < 0 or position > MAX_POSITION: # Outside the grid
            return False
        if position % 2 == 1: # Between Tiles
            return False
        pos_x, pos_y = position_coordinates(position)
        if self.grid[pos_x,pos_y,player] != 0: # Tile unavailable
            return False
        if pos_x%2 == 1: # On top of basis
            return check_basis(pos_x,pos_y,player,tile)
        return True

In [ ]:
test = Tile("Red","Bird")